In [17]:
import numpy as np
import pandas as pd
import plotly.express as px
import random as rd
import json
from scipy.stats import truncnorm

In [30]:
def get_norm_dist(mean=0.5, sd=0.25, low=0, upp=1):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

def generate_prob(n):
    prob = get_norm_dist()
    return prob.rvs(n)

def interaction(probability):
    rand = np.random.rand()
    if(rand <= probability):
        return True
    return False

In [41]:
class Simulation():
    def __init__(self, nodes, timesteps, radius):
        self.population = nodes
        self.susceptibility = generate_prob(nodes)
        self.time = timesteps*4
        self.radius = radius
        
    def generate_walk(self):
        self.generate_nodes()
        for t in range(self.time):
            for j in range(self.population):
                x = rd.randint(0,2)
                y = rd.randint(0,2)
                if x == 1:
                    if self.x[j]+2<=50:
                        self.x[j]+=2
                elif x == 2:
                    if self.x[j]-2>=0:
                        self.x[j]-=2
                if y == 1:
                    if self.y[j]+2<=50:
                        self.y[j]+=2
                elif y == 2:
                    if self.y[j]-2>=0:
                        self.y[j]-=2
            data = {'id': np.arange(self.population), 'timestep': np.ones(self.population)*t, 'x_pos': self.x, 'y_pos': self.y}
            self.df = pd.concat([self.df, pd.DataFrame(data)])
        self.df.reset_index()
            
    def generate_nodes(self):
        self.x = np.random.rand(self.population)*50
        self.y = np.random.rand(self.population)*50
        data = {'id': np.arange(self.population), 'timestep':np.zeros(self.population), 'x_pos': self.x, 'y_pos': self.y}
        self.df = pd.DataFrame(data)
        
    def spread_contagion(self):
        patient0 = rd.randint(0, self.population - 1)
        self.n_status = np.array(["Susceptible" for x in range(self.population)])
        self.n_status[patient0] = "Infected"
        spread = np.copy(self.n_status)
        #for t in range(self.time-1):
            #for node in range(self.population)
            
    def view_df(self):
        return self.df
    
    def run(self):
        self.generate_nodes()
        self.generate_walk()

In [42]:
s = Simulation(50, 60, 5)
s.spread_contagion()

In [43]:
px.scatter(s.view_df(), x="x_pos", y="y_pos", animation_frame="timestep", animation_group="id", color="id", range_x=[0,50], range_y=[0,50])